[View as slides](https://nbviewer.jupyter.org/format/slides/github/lutostag/talks/blob/master/python/Decorators.ipynb#/)

<center><h1>@decorators</h1></center>
  
Greg Lutostanski

[github.com/lutostag](https://github.com/lutostag)

Software Developer
    
  
![Canonical](https://assets.ubuntu.com/v1/5d6da5c4-logo-canonical-aubergine.svg)

# Decorators
* wrappers for functions, classes, or methods
* more functionality
* defined as functions or classes
* called with the "@" symbol

In [2]:
def decorator(func):
    return func

In [3]:
@decorator
def method():
    pass

In [ ]:
class Thing:
    @property
    def attribute(self):
        return "Makes a function a property"
    
    @classmethod
    def classy(cls):
        return "Runs without an instantiation"
    
    @staticmethod
    def static(arg):
        return "Runs without any class or instantiation"
    
# realpython.com/blog/python/instance-class-and-static-methods-demystified/

In [45]:
def fibonacci(n):
    if n <= 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

fibonacci(35)

9227465

In [46]:
from functools import lru_cache

@lru_cache(maxsize=50)
def fibonacci(n):
    if n <= 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

fibonacci(35)

9227465

We know a bit of how to use them

# What are some other uses?
* retries
* cache
* logging

locking, deprecating, disabling, singleton, authentication, authorization, timing

More at [github.com/lord63/awesome-python-decorator](https://github.com/lord63/awesome-python-decorator)

In [4]:
def multiply_by_three(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) * 3
    return wrapper

In [5]:
@multiply_by_three
def fake_add(a, b):
    return a + b

print(fake_add(1, 2))

9


In [6]:
# same as
def fake_add(a, b):
    return a + b

fake_add = multiply_by_three(fake_add)

print(fake_add(1, 2))

9


In [7]:
def multiply_by_three(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs) * 3
    return wrapper


@multiply_by_three
def fake_add(a, b):
    return a + b

In [8]:
# what about arguments to the decorator?

def multiply_by(number):
    def outer_wrapper(func):
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs) * number
        return wrapper
    return outer_wrapper

@multiply_by(4)
def fake_add(a, b):
    return a + b

print(fake_add(1, 3))

16


In [9]:
# can we stack them?

def logging(string):
    def outer_wrapper(func):
        def wrapper(*args, **kwargs):
            print(string)
            return func(*args, **kwargs)
        return wrapper
    return outer_wrapper

@logging('first')
@logging('second')
def something():
    print('yes we can!')
    
something()

first
second
yes we can!


# Gotchas

In [10]:
# Always use functools.wraps

def logging(string):
    def outer_wrapper(func):
        def wrapper(*args, **kwargs):
            print(string)
            return func(*args, **kwargs)
        return wrapper
    return outer_wrapper

@logging('one')
def something():
    """Docstring"""
    print(5)

print(something.__doc__)

None


In [11]:
from functools import wraps

def logging_better(string):
    def outer_wrapper(func):
        @wraps(func) # yes, using a decorator to write a decorator...
        def wrapper(*args, **kwargs):
            print(string)
            return func(*args, **kwargs)
        return wrapper
    return outer_wrapper

@logging_better('one')
def something():
    """Docstring"""
    print(5)
    
print(something.__doc__)

Docstring


In [12]:
# Evaluated at runtime, so don't redefine what it is...

# Extras

In [19]:
# decorator used on a class
from functools import total_ordering

@total_ordering
class Person(object):
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number

    def __eq__(self, other):
        return self.name == other.name and self.id_number == self.id_number

    def __lt__(self, other):
        return (self.name, self.id_number) < (other.name, other.id_number)
    
print(Person('John', 4) > Person('Sandy', 5))

False


In [14]:
# decorator written as a class

In [15]:
# decorator passing args to the wrapped function

# be careful!
# the ordering of args passed to wrapped functions is surprising at first

<center><h1>Thanks!</h1></center>

slides @ [github.com/lutostag/talks](https://github.com/lutostag/talks)

idea for example from [FreshBooks DevBlog](https://www.freshbooks.com/developers/blog/logging-actions-with-python-decorators-part-i-decorating-logged-functions)

Further reading:
* https://github.com/lord63/awesome-python-decorator
* https://wiki.python.org/moin/PythonDecorators
* https://www.python.org/dev/peps/pep-0318/
* https://www.artima.com/weblogs/viewpost.jsp?thread=240845